In [57]:
import json

class SierraQueryBuilder:
    def __init__(self):
        self.queries = []
        self.current_query = None
        self.last_was_operator = False

    def start_query(self, record_type, field_tag):
        if self.current_query is not None:
            raise ValueError("Previous query not ended. Use end_query to finish.")
        if self.last_was_operator:
            self.last_was_operator = False
        # self.current_query = {"target": {"record_type": record_type, "field_tag": field_tag}, "expr": []}
        self.current_query = {
            "target": {
                "record": {
                    "type": record_type
                },
                "field": {
                  "tag": field_tag
                },
            },
            "expr": []
        }
        return self

    def add_expression(self, op, operands):
        if self.current_query is None:
            raise ValueError("No active query. Use start_query to begin.")
        if not isinstance(operands, list):
            operands = [operands]
        expression = {"op": op, "operands": operands}
        # The line below has been commented out to remove the restriction
        # if self.current_query["expr"] and isinstance(self.current_query["expr"][-1], str):
        #     raise ValueError("Must add logical operator before adding another expression to the same query.")
        self.current_query["expr"].append(expression)
        return self

    def add_logical_operator(self, operator):
        if operator not in ['and', 'or']:
            raise ValueError("Operator must be 'and' or 'or'.")
        if self.current_query and not isinstance(self.current_query["expr"][-1], str):
            self.current_query["expr"].append(operator)
        elif not self.queries or self.last_was_operator:
            raise ValueError("Cannot add a logical operator at this point.")
        else:
            self.queries.append(operator)
            self.last_was_operator = True
        return self

    def end_query(self):
        if self.current_query is None:
            raise ValueError("No active query to end.")
        if isinstance(self.current_query["expr"][-1], str):
            raise ValueError("Cannot end query after a logical operator. Add another expression.")
        # if len(self.current_query["expr"]) == 1:
        #     self.current_query["expr"] = self.current_query["expr"][0]
        self.queries.append(self.current_query)
        self.current_query = None
        return self

    def build(self):
        if self.current_query is not None:
            raise ValueError("Query not ended. Use end_query to finish.")
        if self.last_was_operator:
            raise ValueError("Query structure ended with a logical operator.")
        return {"queries": self.queries}

    def __str__(self):
        return json.dumps(self.build(), indent=2)


In [58]:
# I don't know if we want to support this madness or not :-)

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['moby dick']) \
    .add_logical_operator('or') \
        .start_query(record_type='bib', field_tag='a') \
            .add_expression(op='not equal', operands=['melville']) \
        .end_query() \
        .add_logical_operator('and') \
        .start_query(record_type='bib', field_tag='t') \
            .add_expression(op='equal', operands=['moby dick']) \
        .end_query() \
    .end_query() \

print(query)

TypeError: 'NoneType' object is not subscriptable

In [59]:
# Example 1: A simple query with a single, simple expression
# A simple query for Bibliographic title equal to 'birds of america'

# {
#   "target": {
#     "record": {"type": "bib"},
#     "field": {"tag": "t"}
#   },
#   "expr": {
#     "op": "equals",
#     "operands": ["birds of america"]
#   }
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['birds of america']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record": {
          "type": "bib"
        },
        "field": {
          "tag": "t"
        }
      },
      "expr": [
        {
          "op": "equals",
          "operands": [
            "birds of america"
          ]
        }
      ]
    }
  ]
}


In [60]:
# Example 2: A compound query consisting of two simple queries
# A compound query for Bibliographic title equal to 'strangers on a train' and Bibliographic author has 'highsmith'

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "t"}
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["strangers on a train"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "a"}
#       },
#       "expr": {
#         "op": "has",
#         "operands": ["highsmith"]
#       }
#     }
#   ]
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['strangers on a train']) \
    .end_query() \
    .add_logical_operator('and') \
    .start_query(record_type='bib', field_tag='a') \
    .add_expression(op='has', operands=['highsmith']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record": {
          "type": "bib"
        },
        "field": {
          "tag": "t"
        }
      },
      "expr": [
        {
          "op": "equals",
          "operands": [
            "strangers on a train"
          ]
        }
      ]
    },
    "and",
    {
      "target": {
        "record": {
          "type": "bib"
        },
        "field": {
          "tag": "a"
        }
      },
      "expr": [
        {
          "op": "has",
          "operands": [
            "highsmith"
          ]
        }
      ]
    }
  ]
}


In [61]:
# Example 3: A simple query with a compound expression
# A simple query for Bibliographic title equal to 'moby dick' and has 'whale'

# {
#   "target": {
#     "record": {"type": "bib"},
#     "field": {"tag": "t"}
#   },
#   "expr": [
#     {
#       "op": "equals",
#       "operands": ["moby dick"]
#     },
#     "and",
#     {
#       "op": "has",
#       "operands": ["whale"]
#     }
#   ]
# }

query = SierraQueryBuilder()
query.start_query(record_type='bib', field_tag='t') \
    .add_expression(op='equals', operands=['moby dick']) \
    .add_logical_operator('and') \
    .add_expression(op='has', operands=['whale']) \
    .end_query()

print(query)

{
  "queries": [
    {
      "target": {
        "record": {
          "type": "bib"
        },
        "field": {
          "tag": "t"
        }
      },
      "expr": [
        {
          "op": "equals",
          "operands": [
            "moby dick"
          ]
        },
        "and",
        {
          "op": "has",
          "operands": [
            "whale"
          ]
        }
      ]
    }
  ]
}


In [ ]:
# TODO: WHA?!

# Example 4: A soft-linked record type
# A query for Registered Session Summary starts with "gardening lecture"

# {
#   "target": {
#     "record": {
#       "type": "section",
#       "relationType": "soft",
#       "relationTag": "3"
#     },
#     "field": {"tag": "s"}
#   },
#   "expr": {
#     "op": "starts_with",
#     "operands": ["gardening lecture"]
#   }
# }

# query = SierraQueryBuilder()
# query.start_query(record_type='section', field_tag='s') \
#     .add_expression(op='equals', operands=['moby dick']) \
#     .add_logical_operator('and') \
#     .add_expression(op='has', operands=['whale']) \
#     .end_query()

# print(query)

In [ ]:
# TODO: What have we gotten ourselves into

# Example 5: A compound query illustrating syntax for variable-length, fixed-length, and special fields
# A compound query for Bibliographic title has 'mockingbird' and Bibliographic MARC Tag 24501|c has "lee" and Bibliographic material type equal to 'a' and Bibliographic country equal to 'xxu' and Order paid date equals yesterday

# In this query, there are expressions targeting the following field types:

#     a variable-length field using a non-MARC field tag (title / field tag "t")
#     a variable-length field using a MARC field tag with indicators and a subfield (MARC 24501|c)
#     a fixed-length field (material type / ff #30)
#     a special field (MARC 008 'country' subfield / special field #268)
#     a record property (Order record PAID date / record property #79003)

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "field": {"tag": "t"
#         }
#       },
#       "expr": {
#         "op": "has",
#         "operands": ["mockingbird"]
#       }
#     },
#     "and",
#     {
# 	"target": {
# 	"record": {"type": "bib"},
# 	 "field": {"marcTag": "245",
# 		   "ind1": "0",
# 		   "ind2": "1",
# 		   "subfields": "c"
#        }
#      },
#      "expr": {
# 	"op": "has",
#        "operands": ["lee"]
#      }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "id": 30
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["a"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "bib"},
#         "specialField": 268
#       },
#       "expr": {
#         "op": "equals",
#         "operands": ["xxu"]
#       }
#     },
#     "and",
#     {
#       "target": {
#         "record": {"type": "order"},
#         "id": 79003
#       },
#       "expr": {
#         "op": "yesterday",
#         "operands": [""]
#       }
#     }
#   ]
# }				



In [62]:
# Example 6: A query targeting data stored in a barcode field
# OpenA compound query for Item Barcode in ("23100 10850 0098","23100108500101. ","23100108500114")

# Note that Create Lists searches record data when searching for barcodes, and does not search the normalized data stored in the barcode index. Create Lists does not find a barcode unless the data is entered exactly as it appears in the record (including all spaces and punctuation that may be stored in the record's barcode field, and that are typically removed during the indexing process).

# {
#   "queries": [
#     {
#       "target": {
#         "record": {"type": "item"},
#         "field": {"tag": "b"}
#       },
#       "expr": [
#         {
#           "op": "in",
#           "operands": [
#             "23100 10850 0098",
#             "23100108500101. ",
#             "23100108500114"
#           ]
#         }
#       ]
#     }
#   ]
# }				

barcodes = [
    "A000004722146",
    "A000059392227",
    "1517118-1001",
    "8701861246010",
    "0944265058015"
]

query = SierraQueryBuilder()
query.start_query(record_type='item', field_tag='b') \
    .add_expression(op='in', operands=barcodes) \
    .end_query()
print(query)


{
  "queries": [
    {
      "target": {
        "record": {
          "type": "item"
        },
        "field": {
          "tag": "b"
        }
      },
      "expr": [
        {
          "op": "in",
          "operands": [
            "A000004722146",
            "A000059392227",
            "1517118-1001",
            "8701861246010",
            "0944265058015"
          ]
        }
      ]
    }
  ]
}
